---

## Approach

---

Terry stops derive their origin from the Supreme Court case, Terry v. Ohio in which a police officer stopped three suspects he found acting suspicious and found two with firearms. With the fourth Amendment's right against "unreasonable searches and seizures" the court argued in an eight-to-one decision that the officer had probable cause to fear for his safety and the stop-and-search was considered legal. 

Given the prevalence of anti bias training for police forces all over the country, the need for a data driven look at what aspects of policing show bias is important. Although an officer is allowed to use their experience and judgment to determine the correct course of action, when confronted with someone they deem suspicious, an officer's unconscious or conscious bias can preemptively lead to an undue arrest. 

Using Terry Stop data from the Seattle Police Department publicly available online database I intend to perform a binary classification analysis on the stops made that ended in an arrest. The individual features like, age, sex, race, and location will be looked at for their importance in determining whether an arrest was made or not. 

---

# Import Modules & Read Data

---

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',200)
pd.set_option('display.max_columns', 70)
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestRegressor, ExtraTreesRegressor
from pickle import dump
import xgboost
from xgboost import XGBClassifier

In [2]:
cat_df = pd.read_csv('CSV/cleaned_terry_stops_v2.csv')

---

### Correlation Matrix

In [ ]:
fig, ax = plt.subplots(figsize=(40,40))
corr = df.corr()
sns.heatmap(corr, cmap='coolwarm')
plt.show()

---

### Feature Choice

In [3]:
features_df = cat_df.filter(['call_type_1',
                             'call_type_0',
                             'weapon_Blade',
                             'beat_M',
                             'beat_U',
                             'beat_K',
                             'beat_E',
                             'beat_F',
                             'off_race_White',
                             'subj_perceived_race_White',
                             'subj_perceived_race_Black or African American',
                             'off_age',
                             'date'], axis=1)

---

# Train/Test/Split

In [4]:
from sklearn.model_selection import train_test_split

y = cat_df['stop_resolution']
# X = cat_df.drop(['stop_resolution'], axis=1)
X = features_df

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

---

# SMOTE oversampling to account for imbalance in target variable

In [5]:
cat_df.stop_resolution.value_counts()

0    19473
1     9057
Name: stop_resolution, dtype: int64

In [6]:
# Import and fit SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='minority', random_state=23)
X_train_SMOTE, y_train_SMOTE = smote.fit_sample(X_train, y_train)

Using TensorFlow backend.


---

# Baseline model

In [7]:
logreg = LogisticRegression(multi_class='ovr')
logreg.fit(X_train_SMOTE,y_train_SMOTE)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [8]:
y_pred_base = logreg.predict(X_test)

In [9]:
# F1 for test predictions
print("F1:",metrics.f1_score(y_test, y_pred_base))
print("Prec:",metrics.precision_score(y_test, y_pred_base))

F1: 0.4632473048023522
Prec: 0.3710099424385139


In [10]:
max(y_test.mean(), 1 - y_test.mean())

0.6775550259357914

---

# Parameter grids to feed into GridsearchCV

In [11]:
# Logistic regression parameter grid
log_param_grid = { 
    'class_weight': ['balanced','None'],
    'solver': ['sag','saga','liblinear', 'lbfgs'],
    'fit_intercept': ['True', 'False'],
    'intercept_scaling': [1,.01,.001,.0001],
    'C': [1,0.01,0.1,0.0001],
    'tol': [0.01,0.001,0.0001],
    'max_iter': [50,100,150,200]
}

In [14]:
# Random forest parameter grid
rfc_param_grid = { 
    'class_weight': ['balanced','balanced_subsample'],
    'n_estimators': [350, 400, 450],
    'min_impurity_decrease': [.0001,.00001],
    'max_depth': [30, 40, 50]
}

---

# Random Forest Model

In [15]:
rfc = RandomForestClassifier()
grid_rfc = GridSearchCV(rfc, rfc_param_grid, cv = 3, scoring = 'precision', verbose = 1)
grid_rfc.fit(X_train,y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 108 out of 108 | elapsed:  6.3min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [16]:
# Print best parameters and estimator
print(grid_rfc.best_params_)
print(grid_rfc.best_estimator_)

{'class_weight': 'balanced', 'max_depth': 40, 'min_impurity_split': 1e-05, 'n_estimators': 350}
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=40, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=1e-05,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=350,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [17]:
# Predict on the test set
y_pred_grid=grid_rfc.best_estimator_.predict(X_test)

Get F1, accuracy, precision, and recall scores

In [18]:
# Get F1, accuracy, precision, and recall scores
print("F1:",metrics.f1_score(y_test, y_pred_grid))
print("Acc:",metrics.accuracy_score(y_test, y_pred_grid))
print("Prec:",metrics.precision_score(y_test, y_pred_grid))
print("Rec:",metrics.recall_score(y_test, y_pred_grid))

F1: 0.4616283599532528
Acc: 0.6125052572550119
Prec: 0.41813690896259703
Rec: 0.5152173913043478


In [21]:
# Plot feature importance
feat_importances = pd.Series(grid_rfc.best_estimator_.feature_importances_, index=X.columns)
df_feat_importances = pd.DataFrame(feat_importances)
df_feat_importances.sort_values(by=[0], ascending=False)

,0
off_age,0.556938
date,0.178606
call_type_1,0.036207
call_type_0,0.030508
off_race_White,0.030083
weapon_Blade,0.026195
subj_perceived_race_White,0.026116
subj_perceived_race_Black or African American,0.025894
beat_K,0.020951
beat_E,0.019318


---

# Logistic Regression Model

In [8]:
logreg = LogisticRegression(multi_class='ovr')
grid_logreg=GridSearchCV(logreg,log_param_grid,cv=3,scoring='f1',verbose=1)
grid_logreg.fit(X_train_SMOTE,y_train_SMOTE)

Fitting 3 folds for each of 3072 candidates, totalling 9216 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'True'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'True'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'T

/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'True'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'True'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'True'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-pac

/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'True'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'True'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'True'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-pac

/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'True'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'True'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'True'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-pac

...

/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'False'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'False'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'False'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-

/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'False'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'False'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'False'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-

/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'False'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'False'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'False'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-

/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: class_weight must be dict, 'balanced', or None, got: 'None'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: class_weight must be dict, 'balanced', or None, got: 'None'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: invalid literal for int() with base 10: 'False'

  FitFailedWarning)
/Users/kevinmacmat/opt/anaco

GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='ovr',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 0.01, 0.1, 0.0001],
                         'class_weight': ['balanced', 'None'],
                         'fit_intercept': ['True', 'False'],
                         'intercept_scaling': [1, 0.01, 0.001, 0.0001],
                         'max_iter': [50, 100, 150, 200],
                         'solver': ['sag', 's

In [9]:
# Predict on the test set
log_y_pred_grid=grid_logreg.best_estimator_.predict(X_test)

In [10]:
# Print best parameters and best estimator
print(grid_logreg.best_params_)
print(grid_logreg.best_estimator_)

{'C': 1, 'class_weight': 'balanced', 'fit_intercept': 'True', 'intercept_scaling': 0.01, 'max_iter': 50, 'solver': 'liblinear', 'tol': 0.0001}
LogisticRegression(C=1, class_weight='balanced', dual=False,
                   fit_intercept='True', intercept_scaling=0.01, l1_ratio=None,
                   max_iter=50, multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)


In [11]:
# Print scores for F1, accuracy, precision, and recall
print("F1:",metrics.f1_score(y_test, log_y_pred_grid))
print("Acc:",metrics.accuracy_score(y_test, log_y_pred_grid))
print("Prec:",metrics.precision_score(y_test, log_y_pred_grid))
print("Rec:",metrics.recall_score(y_test, log_y_pred_grid))

F1: 0.4763961740203641
Acc: 0.5241833730548157
Prec: 0.36920133907221425
Rec: 0.671304347826087


In [12]:
# Get coefficients
import statsmodels.api as sm
model = sm.Logit(cat_df.stop_resolution, cat_df[['call_type_1',
                             'call_type_0',
                             'weapon_Blade',
                             'beat_M',
                             'beat_U',
                             'beat_K',
                             'beat_E',
                             'beat_F',
                             'off_race_White',
                             'subj_perceived_race_White',
                             'subj_perceived_race_Black or African American',
                             'off_age',
                             'date']])
result = model.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.605659
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        stop_resolution   No. Observations:                28530
Model:                          Logit   Df Residuals:                    28517
Method:                           MLE   Df Model:                           12
Date:                Fri, 29 May 2020   Pseudo R-squ.:                 0.03085
Time:                        10:41:04   Log-Likelihood:                -17279.
converged:                       True   LL-Null:                       -17829.
Covariance Type:            nonrobust   LLR p-value:                5.918e-228
=================================================================================================================
                                                    coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
call_type_1                                       1.0143      0.053     19.017      0.000       0.910       1.119
call_type_0                                       0.9796      0.049     19.999      0.000       0.884       1.076
weapon_Blade                                      0.8304      0.055     14.971      0.000       0.722       0.939
beat_M                                            0.3762      0.043      8.653      0.000       0.291       0.461
beat_U                                           -0.3008      0.066     -4.530      0.000      -0.431      -0.171
beat_K                                            0.0845      0.046      1.833      0.067      -0.006       0.175
beat_E                                            0.1939      0.044      4.381      0.000       0.107       0.281
beat_F                                           -0.2383      0.062     -3.845      0.000      -0.360      -0.117
off_race_White                                    0.1662      0.033      5.046      0.000       0.102       0.231
subj_perceived_race_White                        -0.1422      0.040     -3.557      0.000      -0.221      -0.064
subj_perceived_race_Black or African American    -0.0111      0.042     -0.264      0.792      -0.093       0.071
off_age                                          -0.0169      0.002    -10.494      0.000      -0.020      -0.014
date                                             -0.0006    4.2e-05    -14.312      0.000      -0.001      -0.001
=================================================================================================================
"""

---

# Conclusion

After running multiple models, the prediction metrics did not turn out to be substantially predictive. Features like beats with high numbers of stops, officer age, stops containing a blade featured prominently in multiple iterations of models. While the model itself did not hold much predictive power the exploratory data analysis provided insight into disparities of stops according to race and location of stops corresponding with areas of high displacement risk. For next steps I would like to engineer more features, for example whether a stop was made during day or night, what month, and whether a stop was made on a weekday or weekend.<br>
<br>
Below are the scores for my model iterations:<br>
<br>
Log Regression #1<br>
F1: 0.458<br> 
Precision: 0.416<br>
<br>
Log Regression #2<br>
F1: 0.457<br>
Precision: 0.381<br>
<br>
Log Regression #3<br>
F1: 0.476<br>
Precision: 0.369<br>
<br>  
RFC #1<br>
F1: 0.438<br>
Precision: 0.400<br>
<br>
RFC #2<br>
F1: 0.452<br>
Precision: 0.425<br>
<br>
RFC #3<br>
F1: 0.461<br>
Precision: 0.418<br>

